In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import lightgbm as lgb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization

In [33]:
df = pd.read_csv('/content/sample_data/corona_tested_individuals_ver_006.english.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-04-30,0,0,0,0,0,negative,None,female,Other
1,2020-04-30,1,0,0,0,0,negative,None,female,Other
2,2020-04-30,0,1,0,0,0,negative,None,male,Other
3,2020-04-30,1,0,0,0,0,negative,None,female,Other
4,2020-04-30,1,0,0,0,0,negative,None,male,Other


In [35]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['corona_result']= label_encoder.fit_transform(df['corona_result'])
df['age_60_and_above']= label_encoder.fit_transform(df['age_60_and_above'])
df['gender']= label_encoder.fit_transform(df['gender'])
df['test_indication']= label_encoder.fit_transform(df['test_indication'])


df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-04-30,0,0,0,0,0,0,1,1,2
1,2020-04-30,1,0,0,0,0,0,1,1,2
2,2020-04-30,0,1,0,0,0,0,1,2,2
3,2020-04-30,1,0,0,0,0,0,1,1,2
4,2020-04-30,1,0,0,0,0,0,1,2,2


In [36]:
df.drop('test_date',axis='columns', inplace=True)

In [37]:
# view summary of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278848 entries, 0 to 278847
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   cough                278848 non-null  object
 1   fever                278848 non-null  object
 2   sore_throat          278848 non-null  object
 3   shortness_of_breath  278848 non-null  object
 4   head_ache            278848 non-null  object
 5   corona_result        278848 non-null  int64 
 6   age_60_and_above     278848 non-null  int64 
 7   gender               278848 non-null  int64 
 8   test_indication      278848 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 19.1+ MB


In [38]:
# check the distribution of the target variable
df['corona_result'].value_counts()

0    260227
2     14729
1      3892
Name: corona_result, dtype: int64

In [39]:
df=df.mask(df.eq('None')).dropna()



In [40]:
df['cough'] = pd.to_numeric(df['cough'])
df['fever'] = pd.to_numeric(df['fever'])
df['sore_throat'] = pd.to_numeric(df['sore_throat'])
df['head_ache'] = pd.to_numeric(df['head_ache'])
df['shortness_of_breath'] = pd.to_numeric(df['shortness_of_breath'])


In [41]:
X = df[['cough','fever','sore_throat','shortness_of_breath','head_ache','age_60_and_above','gender','test_indication']]
y = df['corona_result']


In [42]:
X.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication
0,0,0,0,0,0,1,1,2
1,1,0,0,0,0,1,1,2
2,0,1,0,0,0,1,2,2
3,1,0,0,0,0,1,1,2
4,1,0,0,0,0,1,2,2


In [43]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: corona_result, dtype: int64

In [44]:
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [45]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(num_leaves= 20, min_data_in_leaf= 4, feature_fraction= 0.2, bagging_fraction= 0.8, bagging_freq= 5, learning_rate= 0.05,verbose=1)
clf.fit(X_train, y_train)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.2,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=4,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=20,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0, verbose=1)

In [46]:
y_pred=clf.predict(X_test)

y_pred[0]


0

In [47]:
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9424


In [48]:
y_pred_train = clf.predict(X_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

Training-set accuracy score: 0.9422


In [49]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Training set score: 0.9422
Test set score: 0.9424


In [50]:
# view confusion-matrix
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[77852     0   172]
 [ 1171     0    20]
 [ 3455     0   909]]

True Positives(TP) =  77852

True Negatives(TN) =  0

False Positives(FP) =  0

False Negatives(FN) =  1171


In [51]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     78024
           1       0.00      0.00      0.00      1191
           2       0.83      0.21      0.33      4364

    accuracy                           0.94     83579
   macro avg       0.59      0.40      0.43     83579
weighted avg       0.92      0.94      0.92     83579



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
!pip install flask_ngrok

In [ ]:
# flask_ngrok_example.py
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request, jsonify
from collections import defaultdict
import json

answer_dict = {'YES':1,'NO':0,'Male':1,'Female':0}
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/",methods=['GET', 'POST'])
def hello():
    data = request.json
    print(data)
    features = []
    
    for obj in data:
      if obj.get('question') != 'Address':
        features.append(answer_dict[obj['selected_answer'][0]['answer_choice']])
    answer = clf.predict([features])
    print(answer)
    return json.dumps([{'answer':str(answer[0])}])

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f66d38887935.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/May/2021 14:52:50] "POST / HTTP/1.1" 200 -


[{'question': 'Do you have fever ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Are you over 60 years of age ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'SEX ?', 'selected_answer': [{'answer_choice': 'Female'}]}, {'question': 'Do you have cough ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have shortness of breath ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have sore throat ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have headache ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Was you in contact with covid19 infected person ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Address', 'selected_answer': [{'answer_choice': '2515, Wadekar lean, Chandrashekhar Chowk, Rangargalli, Sangamner, Maharashtra 422605, India'}]}]
[0]


...

[{'question': 'Do you have fever ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Are you over 60 years of age ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'SEX ?', 'selected_answer': [{'answer_choice': 'Female'}]}, {'question': 'Do you have cough ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have shortness of breath ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have sore throat ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have headache ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Was you in contact with covid19 infected person ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Address', 'selected_answer': [{'answer_choice': '2515, Wadekar lean, Chandrashekhar Chowk, Rangargalli, Sangamner, Maharashtra 422605, India'}]}]
[0]


127.0.0.1 - - [28/May/2021 15:05:47] "POST / HTTP/1.1" 200 -


[{'question': 'Do you have fever ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Are you over 60 years of age ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'SEX ?', 'selected_answer': [{'answer_choice': 'Female'}]}, {'question': 'Do you have cough ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have shortness of breath ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have sore throat ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Do you have headache ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Was you in contact with covid19 infected person ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Address', 'selected_answer': [{'answer_choice': '2515, Wadekar lean, Chandrashekhar Chowk, Rangargalli, Sangamner, Maharashtra 422605, India'}]}]
[0]


127.0.0.1 - - [28/May/2021 15:07:23] "POST / HTTP/1.1" 200 -


[{'question': 'Do you have fever ?', 'selected_answer': [{'answer_choice': 'NO'}]}, {'question': 'Are you over 60 years of age ?', 'selected_answer': [{'answer_choice': 'YES'}]}, {'question': 'SEX ?', 'selected_answer': [{'answer_choice': 'Male'}]}, {'question': 'Do you have cough ?', 'selected_answer': [{'answer_choice': 'YES'}]}, {'question': 'Do you have shortness of breath ?', 'selected_answer': [{'answer_choice': 'YES'}]}, {'question': 'Do you have sore throat ?', 'selected_answer': [{'answer_choice': 'YES'}]}, {'question': 'Do you have headache ?', 'selected_answer': [{'answer_choice': 'YES'}]}, {'question': 'Was you in contact with covid19 infected person ?', 'selected_answer': [{'answer_choice': 'YES'}]}, {'question': 'Address', 'selected_answer': [{'answer_choice': '2515, Wadekar lean, Chandrashekhar Chowk, Rangargalli, Sangamner, Maharashtra 422605, India'}]}]
[2]
